In [29]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


In [27]:
from tqdm.auto import tqdm

In [ ]:
model_path = "./finetuned_kcbert_large"
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [5]:
test_df = pd.read_excel("아이스크림에듀 라벨링 데이터 (1~1000).xlsx")

In [6]:
test_df

,후기,라벨링
0,구글활용법 연수를 통해 구글활용도에 대한 이해를 높이고 소소한 팁들을 많이 알게 되...,1
1,평소 정리의 힘을 중요하게 생각해서 신청한 연수입니다. 소소한 꿀팁을 얻을 수 있어...,1
2,"학급에 존재할 수 있는 다양한 사례에 맞춘 교수, 학습법을 강의 내용으로 구성하여 ...",1
3,수업 시간에 활용할 수 있는 간편하고 유익한 웹 설명 좋았습니다.,1
4,연수를 통해 다양한 진로 교육 방법을 사례를 통해 배울 수 있어서 매우 유익했습니다!,1
...,...,...
990,연수 잘 들었습니다. 열심히 활용하도록 하겠습니다.,1
991,아이들과 교실에서 꼭 해보고 싶은 활동입니다.,1
992,학급경영에서 3월 첫날부터 5일까지의 놀이와 적절한 인사법.\n\n그리고 학부모와의...,1
993,선생님 말씀이 참 따뜻하고 그림책을 읽어줄 때마다 감동적인 표정과 어투로 읽어줄 때...,1


In [9]:
test_encodings = tokenizer(test_df['후기'].tolist(), truncation=True, padding=True, max_length=300, return_tensors="pt")
test_labels = torch.tensor(test_df['라벨링'].tolist())

In [24]:
dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
loader = DataLoader(dataset, batch_size=16, shuffle=False)

In [ ]:
model.eval()

In [28]:
predictions = []
with torch.no_grad():
    for input_ids, attention_mask, _ in tqdm(loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

100%|██████████| 63/63 [05:14<00:00,  4.99s/it]


In [31]:
true_labels = test_labels.numpy()
predictions = np.array(predictions)

In [32]:
print(classification_report(true_labels, predictions, target_names=['Negative', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.67      0.64      0.66        61
    Positive       0.98      0.98      0.98       934

    accuracy                           0.96       995
   macro avg       0.82      0.81      0.82       995
weighted avg       0.96      0.96      0.96       995

